In [ ]:
class BERTClassifier(nn.Module):
    \"\"\"BERT-tiny classifier optimized for federated learning.\"\"\"

    def __init__(self, model_name: str, num_classes: int = 4, dropout: float = 0.1):
        super().__init__()
        self.model_name = model_name
        self.num_classes = num_classes

        # Load BERT configuration and model
        self.config = AutoConfig.from_pretrained(model_name)
        self.bert = AutoModel.from_pretrained(model_name, config=self.config)

        # Classification head
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.config.hidden_size, num_classes)

        # Initialize classifier weights
        nn.init.normal_(self.classifier.weight, std=0.02)
        nn.init.zeros_(self.classifier.bias)

    def forward(self, input_ids, attention_mask, labels=None):
        # BERT forward pass
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        # Use [CLS] token representation
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)

        # Classification
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {'loss': loss, 'logits': logits, 'hidden_states': outputs.last_hidden_state}

    def get_embeddings(self, input_ids, attention_mask):
        \"\"\"Extract embeddings for drift detection.\"\"\"
        with torch.no_grad():
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            # Return [CLS] token embeddings
            return outputs.pooler_output


def create_model_and_tokenizer(config: Dict[str, Any], device: torch.device):
    \"\"\"Create BERT model and tokenizer.\"\"\"
    model_name = config['model']['model_name']
    num_classes = config['model']['num_classes']
    dropout = config['model']['dropout']

    # Create tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Create model
    model = BERTClassifier(model_name, num_classes, dropout)
    model = model.to(device)

    # Enable mixed precision if GPU available
    if device.type == 'cuda':
        model = model.half()  # Use FP16 for memory efficiency

    return model, tokenizer


print(\"✅ BERT model implementation ready!\")"

## 🤖 BERT Model Implementation

BERT-tiny classifier with GPU optimization for federated learning.

# 🔄 Federated LLM Drift Detection and Recovery System

**A comprehensive standalone implementation for Google Colab with GPU acceleration**

## 📋 System Overview

This notebook implements a sophisticated **multi-level drift detection architecture** for federated learning with BERT-tiny models:

### 🏗️ **Architecture Components**
- **Client-Side Detection**: ADWIN (concept drift) + Evidently (data drift)
- **Server-Side Detection**: MMD statistical test on embedding aggregates  
- **Adaptive Mitigation**: FedAvg → FedTrimmedAvg when drift detected
- **Synthetic Drift Injection**: Vocabulary shift, label noise, distribution shift

### 🎯 **Key Features**
- GPU-optimized for Google Colab (T4/P100/V100)
- Real-time drift monitoring with visual analytics
- Configurable drift scenarios and client heterogeneity
- Production-ready federated learning pipeline

---

## 🚀 Environment Setup and Dependencies

First, let's install all required packages and configure the environment for optimal GPU performance.

In [ ]:
# Install required packages with GPU support
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers[torch]>=4.56.0
!pip install -q datasets>=4.0.0
!pip install -q scikit-learn>=1.7.0
!pip install -q flwr[simulation]>=1.20.0
!pip install -q alibi-detect>=0.12.0
!pip install -q evidently>=0.7.14
!pip install -q river>=0.22.0
!pip install -q nlpaug>=1.1.11
!pip install -q matplotlib seaborn plotly
!pip install -q pandas numpy scipy
!pip install -q pyyaml

print("✅ All dependencies installed successfully!")

In [ ]:
# Configure GPU and environment
import torch
import os
import gc
import warnings
warnings.filterwarnings('ignore')

# Set environment variables for optimal performance
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Check GPU availability and configure
def setup_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"🎮 GPU Available: {torch.cuda.get_device_name(0)}")
        print(f"📊 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        
        # Optimize memory usage
        torch.cuda.empty_cache()
        torch.backends.cudnn.benchmark = True
        
        # Set memory fraction to prevent OOM
        if hasattr(torch.cuda, 'set_per_process_memory_fraction'):
            torch.cuda.set_per_process_memory_fraction(0.9)
            
    else:
        device = torch.device('cpu')
        print("⚠️ No GPU available, using CPU")
    
    return device

device = setup_device()
print(f"🔧 Device configured: {device}")

In [ ]:
# Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import logging
import time
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any, Callable
from collections import defaultdict

# ML and DL imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AutoConfig
from sklearn.metrics import accuracy_score, classification_report
from datasets import load_dataset

# Federated Learning
import flwr as fl
from flwr.simulation import start_simulation
from flwr.common import Context, Parameters, Scalar
from flwr.server.strategy import FedAvg

# Drift Detection
from alibi_detect import MMDDrift
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from river.drift import ADWIN
import nlpaug.augmenter.word as naw

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ All imports loaded successfully!")

## ⚙️ Configuration and Constants

Define all configuration parameters for the federated learning simulation.

In [ ]:
# Global Configuration for Federated Learning Drift Detection System
CONFIG = {
    # Model configuration
    'model': {
        'model_name': 'prajjwal1/bert-tiny',
        'num_classes': 4,
        'max_length': 128,
        'batch_size': 16,
        'learning_rate': 2e-5,
        'num_epochs': 3,
        'warmup_steps': 100,
        'dropout': 0.1
    },

    # Federated learning configuration
    'federated': {
        'num_clients': 10,
        'alpha': 0.5,  # Dirichlet concentration for non-IID
        'min_samples_per_client': 50
    },

    # Drift configuration
    'drift': {
        'injection_round': 25,
        'drift_intensity': 0.3,
        'affected_clients': [2, 5, 8],  # Which clients get drift
        'drift_types': ['label_noise', 'vocab_shift']
    },

    # Drift detection configuration
    'drift_detection': {
        'adwin_delta': 0.002,
        'mmd_p_val': 0.05,
        'mmd_permutations': 100,
        'evidently_threshold': 0.25,
        'trimmed_beta': 0.2,  # For FedTrimmedAvg
    },

    # Simulation configuration
    'simulation': {
        'num_rounds': 50,
        'fraction_fit': 1.0,
        'fraction_evaluate': 1.0,
        'min_fit_clients': 2,
        'min_evaluate_clients': 2,
        'mitigation_threshold': 0.3  # >30% clients reporting drift
    }
}

print("✅ Configuration loaded:")
print(f"📊 Clients: {CONFIG['federated']['num_clients']}")
print(f"🔄 Rounds: {CONFIG['simulation']['num_rounds']}")
print(f"💥 Drift injection: Round {CONFIG['drift']['injection_round']}")
print(f"🎯 Affected clients: {CONFIG['drift']['affected_clients']}")